In [1]:
import os
import json
import pandas as pd
import traceback

In [14]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

In [6]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192",temperature=0.5)

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain 

## implementing callback in groq

In [12]:
from typing import Any, Dict, List

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate

class LoggingHandler(BaseCallbackHandler):
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        if serialized is not None:
            print(f"Chain {serialized.get('name')} started")
        else:
            print("Chain started with no serialized data")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")

callbacks = [LoggingHandler()]
prompt = ChatPromptTemplate.from_template("What is 1 + {number}?")

chain = prompt | model

# Ensure callbacks are configured correctly
chain_with_callbacks = chain.with_config(callbacks=callbacks)

# Invoke the chain with input
chain_with_callbacks.invoke({"number": "2"})

Chain started with no serialized data
Chain ChatPromptTemplate started
Chain ended, outputs: messages=[HumanMessage(content='What is 1 + 2?')]
Chat model started
Chat model ended, response: generations=[[ChatGeneration(text='The answer is 3!', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='The answer is 3!', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 18, 'total_tokens': 25, 'completion_time': 0.005833333, 'prompt_time': 0.001860871, 'queue_time': 0.018615468, 'total_time': 0.007694204}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-b3f14fa2-9181-4cb1-91b3-64ac563559d0-0', usage_metadata={'input_tokens': 18, 'output_tokens': 7, 'total_tokens': 25}))]] llm_output={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 18, 'total_tokens': 25, 'completion_time': 0.005833333, 'prompt_time': 0.001860871, 'queue_time': 0.018615468, 'total_tim

AIMessage(content='The answer is 3!', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 18, 'total_tokens': 25, 'completion_time': 0.005833333, 'prompt_time': 0.001860871, 'queue_time': 0.018615468, 'total_time': 0.007694204}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-b3f14fa2-9181-4cb1-91b3-64ac563559d0-0', usage_metadata={'input_tokens': 18, 'output_tokens': 7, 'total_tokens': 25})

In [17]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [36]:
from langchain_core.prompts import PromptTemplate

In [37]:
template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [39]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template
    )

In [41]:
quiz_chain=LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Chotu chapri\AppData\Local\Temp\ipykernel_12744\2075755995.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [42]:
template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [43]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=template2)

In [44]:
review_chain=LLMChain(llm=model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [45]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [46]:
file_path = r"D:\gen ai\mcq_generator\data.txt"

In [47]:
file_path

'D:\\gen ai\\mcq_generator\\data.txt'

In [48]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [49]:
print(TEXT)

Monkey D. Luffy, commonly known as "Straw Hat Luffy" or simply "Straw Hat",[10] is the founder, captain, and strongest combatant of the increasingly infamous and powerful Straw Hat Pirates.[26][27] He fearlessly pursues the legendary treasure of the late Gol D. Roger in order to become the new Pirate King[28] and reach a further, untold dream (currently known to only his crew, family and closest friends).[29][30] He believes that being the Pirate King means having the most freedom in the world.[31]

Born on Dawn Island, Luffy is the son of Revolutionary leader Monkey D. Dragon[32] and grandson of the Marine hero Monkey D. Garp[33] and as such is a member of the Clan of D. Most of his childhood was shaped by Garp and Curly Dadan,[34] under whose care he befriended and swore brotherhood to the late Portgas D. "Fire Fist" Ace[35] and Sabo.[36] However, he draws his greatest inspiration from the pirate "Red-Haired" Shanks, who (unwittingly) enabled him to eat the Devil Fruit that turned hi

In [51]:
#Serialize the Python dicitonary to JSON-formatted string
RESPONSE_JSON=json.dumps(RESPONSE_JSON)

In [52]:
NUM_QUESTIONS = 4
SUBJECT = "Mathematics"
TONE = "formal"

In [57]:
class LoggingHandler(BaseCallbackHandler):
    def __init__(self):
        self.total_tokens = 0
        self.responses = []
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

        #Assuming response contains token information
        if hasattr(response, 'usage'):
            self.total_tokens += response.usage.get('total_tokens', 0)
            self.responses.append(response)

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        if serialized is not None:
            print(f"Chain {serialized.get('name')} started")
        else:
            print("Chain started with no serialized data")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")
        print(f"Total tokens used: {self.total_tokens}")

In [58]:
logging_handler = LoggingHandler()

In [60]:
response = generate_evaluate_chain.invoke({"text": TEXT, "number": NUM_QUESTIONS, "subject": SUBJECT, "tone": TONE, "response_json": RESPONSE_JSON}, callbacks=[logging_handler])

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Monkey D. Luffy, commonly known as "Straw Hat Luffy" or simply "Straw Hat",[10] is the founder, captain, and strongest combatant of the increasingly infamous and powerful Straw Hat Pirates.[26][27] He fearlessly pursues the legendary treasure of the late Gol D. Roger in order to become the new Pirate King[28] and reach a further, untold dream (currently known to only his crew, family and closest friends).[29][30] He believes that being the Pirate King means having the most freedom in the world.[31]

Born on Dawn Island, Luffy is the son of Revolutionary leader Monkey D. Dragon[32] and grandson of the Marine hero Monkey D. Garp[33] and as such is a member of the Clan of D. Most of his childhood was shaped by Garp and Curly Dadan,[34] under whose care he befriended and swore brotherhood to the late Portgas D. "Fire Fist" Ace[35] and Sabo.[36] However, he draws his greatest inspiration from the pirate "Red-Haired" Shanks, who (unwittingly) enabled him to eat

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Mathematics students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
Here is the quiz in the required format:

{
"1": {
"mcq": "What is the name of the pirate crew founded by Monkey D. Luffy?",
"options": {
"a": "Sunset Pirates",
"b": "Straw Hat Pirates",
"c": "Revolutionary Pirates",
"d": "Marine Pirates"
},
"correct": "b"
},
"2": {
"mcq": "What is Monkey D. Luffy's goal in pursuing the legendary treasure of Gol D. Roger?",
"options": {
"a": "To become the most powerful pirate in the world",
"b": "To become the new Pirate King",
"c": "To find his los

In [61]:
print(response)

{'text': 'Monkey D. Luffy, commonly known as "Straw Hat Luffy" or simply "Straw Hat",[10] is the founder, captain, and strongest combatant of the increasingly infamous and powerful Straw Hat Pirates.[26][27] He fearlessly pursues the legendary treasure of the late Gol D. Roger in order to become the new Pirate King[28] and reach a further, untold dream (currently known to only his crew, family and closest friends).[29][30] He believes that being the Pirate King means having the most freedom in the world.[31]\n\nBorn on Dawn Island, Luffy is the son of Revolutionary leader Monkey D. Dragon[32] and grandson of the Marine hero Monkey D. Garp[33] and as such is a member of the Clan of D. Most of his childhood was shaped by Garp and Curly Dadan,[34] under whose care he befriended and swore brotherhood to the late Portgas D. "Fire Fist" Ace[35] and Sabo.[36] However, he draws his greatest inspiration from the pirate "Red-Haired" Shanks, who (unwittingly) enabled him to eat the Devil Fruit th

In [62]:
print(f"Total tokens used: {logging_handler.total_tokens}")

Total tokens used: 0


In [63]:
quiz = response.get("quiz")

In [72]:
print(quiz)

Here is the quiz in the required format:

{
"1": {
"mcq": "What is the name of the pirate crew founded by Monkey D. Luffy?",
"options": {
"a": "Sunset Pirates",
"b": "Straw Hat Pirates",
"c": "Revolutionary Pirates",
"d": "Marine Pirates"
},
"correct": "b"
},
"2": {
"mcq": "What is Monkey D. Luffy's goal in pursuing the legendary treasure of Gol D. Roger?",
"options": {
"a": "To become the most powerful pirate in the world",
"b": "To become the new Pirate King",
"c": "To find his lost childhood friend",
"d": "To avenge his family"
},
"correct": "b"
},
"3": {
"mcq": "Who is the person who enabled Monkey D. Luffy to eat the Devil Fruit that turned his body into rubber?",
"options": {
"a": "Monkey D. Dragon",
"b": "Curly Dadan",
"c": "Red-Haired" Shanks,
"d": "Portgas D. Ace"
},
"correct": "c"
},
"4": {
"mcq": "What is the symbol of the vow between Monkey D. Luffy and Red-Haired Shanks?",
"options": {
"a": "A golden anchor",
"b": "A red hat",
"c": "A straw hat",
"d": "A pair of gloves"
},

In [78]:
lines = quiz.splitlines()
quiz_without_first_line = "\n".join(lines[1:])

In [83]:
quiz_new = pd.DataFrame(quiz_without_first_line.split("\n"), columns=["quiz"])

In [85]:
quiz_new.to_csv("quiz.csv", index=False)